In [1323]:
# 제 1유형 필요한 라이브러리
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import StandardScaler, MinMaxScaler,LabelEncoder
 

In [1324]:
# 제 2유형 필요한 라이브러리 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score,recall_score,precision_score,roc_auc_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error



In [ ]:
# 제 3유형 필요한 라이브러리 
from scipy.stats import ttest_1samp,ttest_rel,ttest_ind,f_oneway,wilcoxon,ranksums,kruskal,shapiro,bartlett,levene,chisquare,chi2_contingency,pearsonr
from sklearn.linear_model import LinearRegression, LogisticRegression

######### 암기 필요 #########
import statsmodels.api as sm 

In [1326]:
# 제 1유형 
mtcars = pd.read_csv('../mtcars.csv')
mtcars.rename(columns={'Unnamed: 0' :'car'}, inplace=True)

In [1327]:
# 타입 변경 
# cyl -> object
# gear -> object 
cp_mtcars = mtcars.astype({'cyl':'object','gear':'object'})
cp_mtcars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   car     32 non-null     object 
 1   mpg     32 non-null     float64
 2   cyl     32 non-null     object 
 3   disp    32 non-null     float64
 4   hp      32 non-null     int64  
 5   drat    32 non-null     float64
 6   wt      32 non-null     float64
 7   qsec    32 non-null     float64
 8   vs      32 non-null     int64  
 9   am      32 non-null     int64  
 10  gear    32 non-null     object 
 11  carb    32 non-null     int64  
dtypes: float64(5), int64(4), object(3)
memory usage: 3.1+ KB


In [1328]:
# mpg 컬럼 통계값 다 구하기
mean = mtcars['mpg'].mean()
median = mtcars['mpg'].median()
mode = mtcars['mpg'].mode()
min = mtcars['mpg'].min()
max = mtcars['mpg'].max()
skew = mtcars['mpg'].skew() # 왜도
krut = mtcars['mpg'].kurt() # 첨도
std = mtcars['mpg'].std() # 표준편차
var = mtcars['mpg'].var() # 분산 
q3 = mtcars['mpg'].quantile(0.75)
q1 = mtcars['mpg'].quantile(0.25)
iqr = abs(q1-q3)
# 이상치 범위 lower q1-1.5*iqr 
# 이상치 범위 upper q3+1.5*iqr
print('평균:', mean)
print('중앙값:', median)
print('최빈값:', mode)
print('최솟값:', min)
print('최댓값:', max)
print('왜도:', skew)
print('첨도:', krut)
print('포준편차:', std)
print('분산:', var)
print('q3:', q3)
print('q1:', q1)
print('iqr:', iqr)


평균: 20.090624999999996
중앙값: 19.2
최빈값: 0    10.4
1    15.2
2    19.2
3    21.0
4    21.4
5    22.8
6    30.4
Name: mpg, dtype: float64
최솟값: 10.4
최댓값: 33.9
왜도: 0.6723771376290805
첨도: -0.0220062914240855
포준편차: 6.026948052089105
분산: 36.32410282258065
q3: 22.8
q1: 15.425
iqr: 7.375


In [1329]:
mtcars.groupby('cyl')['car'].count()

cyl
4    11
6     7
8    14
Name: car, dtype: int64

In [1330]:
mtcars.groupby(['gear','cyl']).car.count()

gear  cyl
3     4       1
      6       2
      8      12
4     4       8
      6       4
5     4       2
      6       1
      8       2
Name: car, dtype: int64

In [1331]:
help(mtcars.pivot_table)

Help on method pivot_table in module pandas.core.frame:

pivot_table(values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False, sort=True) -> 'DataFrame' method of pandas.core.frame.DataFrame instance
    Create a spreadsheet-style pivot table as a DataFrame.
    
    The levels in the pivot table will be stored in MultiIndex objects
    (hierarchical indexes) on the index and columns of the result DataFrame.
    
    Parameters
    ----------
    values : column to aggregate, optional
    index : column, Grouper, array, or list of the previous
        If an array is passed, it must be the same length as the data. The
        list can contain any of the other types (except list).
        Keys to group by on the pivot table index.  If an array is passed,
        it is being used as the same manner as column values.
    columns : column, Grouper, array, or list of the previous
        If an array is passed, it m

In [1332]:
mtcars.pivot_table(index='gear',columns='cyl',values='car',aggfunc='count')

cyl,4,6,8
gear,,,
3,1.0,2.0,12.0
4,8.0,4.0,NaN
5,2.0,1.0,2.0


In [1333]:
import seaborn as sns
titanic = sns.load_dataset('titanic')[['age','sex','class','fare','survived']]
titanic.head()

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0


In [1334]:
titanic.pivot_table(index=('sex','survived'), columns='class',values='fare',aggfunc='mean')

class                 First     Second      Third
sex    survived                                  
female 0         110.604167  18.250000  19.773093
       1         105.978159  22.288989  12.464526
male   0          62.894910  19.488965  12.204469
       1          74.637320  21.095100  15.579696

In [1335]:
titanic.groupby(['class','survived'])['age'].mean()

class   survived
First   0           43.695312
        1           35.368197
Second  0           33.544444
        1           25.901566
Third   0           26.555556
        1           20.646118
Name: age, dtype: float64

In [1336]:
cond1 = mtcars['hp'] > 205
mtcars['hp'] = np.where(cond1,205,mtcars['hp'])
mtcars[['hp']]

,hp
0,110
1,110
2,93
3,110
4,175
5,105
6,205
7,62
8,95
9,123


In [1337]:
########### scaling 하기 ###########
zscaler = StandardScaler()
mtcars['mpg'] = zscaler.fit_transform(mtcars[['mpg']])

In [1338]:
########## 날짜 ##########
# 테스트 데이터 만들기
df = pd.DataFrame({
  '날짜': ['20240105', '20240105', '20240223', '20240223', '20240312', '20240422', '20230506'],
  '물품': ['A', 'B', 'A', 'B', 'A', 'B', 'A'],
  '판매수': [5, 10, 15, 15, 20, 25, 40],
  '개당수익': [500, 600, 500, 600, 600, 700, 600]
})

df

,날짜,물품,판매수,개당수익
0,20240105,A,5,500
1,20240105,B,10,600
2,20240223,A,15,500
3,20240223,B,15,600
4,20240312,A,20,600
5,20240422,B,25,700
6,20230506,A,40,600


In [1339]:
# 날짜데이터 datetime 형식으로 변환하기
df['날짜'] = pd.to_datetime(df['날짜'])
df.dtypes

날짜      datetime64[ns]
물품              object
판매수              int64
개당수익             int64
dtype: object

In [1340]:
df['year'] = df['날짜'].dt.year


In [1341]:
# 날짜 구간 필터링 
df[df['날짜'].between('2024-01-24 12:25:00','2024-09-24 21:11:50')]


,날짜,물품,판매수,개당수익,year
2,2024-02-23,A,15,500,2024
3,2024-02-23,B,15,600,2024
4,2024-03-12,A,20,600,2024
5,2024-04-22,B,25,700,2024


In [1342]:
round(len(mtcars[mtcars['cyl']==6])/len(mtcars),1)

0.2

In [1343]:
new_df = pd.DataFrame({
  "날짜": ['20220103', '20220105', None, '20230127', '20220203', '20220205', '20230223', '20230325', '20230427', '20230526', '20230603', '20230723'],
  '제품': ['A', 'B', None, 'B', 'A', None, 'A', 'B', 'A', 'B', 'A', 'A'],
  '판매수': [3, None, 5, 10, 10, 10, 15, 15, 20, None, 30, 40],
  '개당수익': [300, 400, 500, 600, 400, 500, 500, 600, 600, 700, 600, 600]
})

In [1344]:
mtcars = pd.read_csv('../mtcars.csv')
mtcars.head()

zscore = StandardScaler()
mtcars_cp = mtcars.copy()
mtcars_cp['hp'] = zscore.fit_transform(mtcars_cp[['hp']])
# q1 = mtcars_cp['hp'].quantile(0.25)
# q3 = mtcars_cp['hp'].quantile(0.75)
# iqr = abs(q1-q3)
# upper = q3+1.5*iqr
# lower = q1-1.5*iqr

len(mtcars_cp[(mtcars_cp['hp']>1.5) | (mtcars_cp['hp'] <-1.5)])

2

In [1345]:
m_scaler = MinMaxScaler()
mtcars_cp2 = mtcars.copy()
mtcars_cp2['mpg'] = m_scaler.fit_transform(mtcars_cp2[['mpg']])
len(mtcars_cp2[mtcars_cp2['mpg']>0.7])

5

In [1346]:
mtcars_cp3 = mtcars.sort_values(by='hp',ascending=False).head(7)
mtcars_cp3.reset_index(inplace=True,drop=True)
hp_7 = mtcars_cp3.iloc[6].hp
mtcars_cp4 = mtcars.copy()
mtcars_cp4['hp'] = np.where(mtcars_cp4['hp']>hp_7,hp_7,mtcars_cp4['hp'])
mtcars_cp4[mtcars_cp4['hp']>=150]['hp'].mean()

186.66666666666666

In [1347]:
mtcars.rename(columns={'Unnamed: 0' : 'car'},inplace=True)
cond = mtcars['car'].str.contains('Merc')
mtcars[cond]['mpg'].mean()

19.014285714285712

In [1348]:
# 결과값 가공 
# final_result = pd.DateFrame({'cust_id':cust_id, 'result': result})
# final_result.to_csv('생년원일_시험번호.csv',index=False)

In [1349]:
# 제 2유형 타이타닉 생존자 분류하기 
import seaborn as sns
df = sns.load_dataset('titanic')
x=df.drop('survived', axis = 1)
y = df['survived']
x_train, x_test , y_train , y_test = train_test_split(x,y,stratify=y, test_size=0.2, random_state=2023)

cust_id = pd.DataFrame(x_test.reset_index())['index']

x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

y_train.columns = ['target']
y_test.columns =['target']

In [1350]:
print(x_train.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 3 to 608
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   pclass       712 non-null    int64   
 1   sex          712 non-null    object  
 2   age          579 non-null    float64 
 3   sibsp        712 non-null    int64   
 4   parch        712 non-null    int64   
 5   fare         712 non-null    float64 
 6   embarked     710 non-null    object  
 7   class        712 non-null    category
 8   who          712 non-null    object  
 9   adult_male   712 non-null    bool    
 10  deck         164 non-null    category
 11  embark_town  710 non-null    object  
 12  alive        712 non-null    object  
 13  alone        712 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(3), object(5)
memory usage: 64.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 3 to 608
Data columns (total 1 columns):
 #   Column  

In [1351]:
print(x_train.isnull().sum())
print(x_test.isnull().sum())
# print(y_train.isnull().sum())

pclass           0
sex              0
age            133
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           548
embark_town      2
alive            0
alone            0
dtype: int64
pclass           0
sex              0
age             44
sibsp            0
parch            0
fare             0
embarked         0
class            0
who              0
adult_male       0
deck           140
embark_town      0
alive            0
alone            0
dtype: int64


In [1352]:
x_train.describe().T

,count,mean,std,min,25%,50%,75%,max
pclass,712.0,2.307584,0.834926,1.00,2.0000,3.0000,3.000,3.0000
age,579.0,29.479568,14.355304,0.42,20.0000,28.0000,38.000,74.0000
sibsp,712.0,0.518258,1.094522,0.00,0.0000,0.0000,1.000,8.0000
parch,712.0,0.372191,0.792341,0.00,0.0000,0.0000,0.000,6.0000
fare,712.0,31.741836,45.403910,0.00,7.8958,14.4542,31.275,512.3292


In [1353]:
print(y_train.head())

     target
3         1
517       0
861       0
487       0
58        1


In [1354]:
x_train.head()

,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
3,1,female,35.0,1,0,53.10,S,First,woman,False,C,Southampton,yes,False
517,3,male,NaN,0,0,24.15,Q,Third,man,True,NaN,Queenstown,no,True
861,2,male,21.0,1,0,11.50,S,Second,man,True,NaN,Southampton,no,False
487,1,male,58.0,0,0,29.70,C,First,man,True,B,Cherbourg,no,True
58,2,female,5.0,1,2,27.75,S,Second,child,False,NaN,Southampton,yes,False


In [1355]:
x_train['embarked'].mode()[0]

'S'

In [1356]:
# x_train : age, embarked, deck, embark_town
# x_train.head()
x_train.drop(columns=['deck','embark_town','class','adult_male','alive'],inplace=True)
x_test.drop(columns=['deck','embark_town','class','adult_male','alive'],inplace=True)
print(x_train.head())
# print(x_train.info())
x_train['age']=x_train['age'].fillna(x_train['age'].median())
x_train['embarked'] = x_train['embarked'].fillna(x_train['embarked'].mode()[0])
# x_train['deck'] = x_train['deck'].fillna(x_train['deck'].mode()[0],inplace=True)
# print('결측치 처리후 :' ,x_train.info())
x_test['age'] = x_test['age'].fillna(x_test['age'].median())
# x_test['deck'] = x_test['deck'].fillna(x_test['deck'].mode()[0],inplace=True)

x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)
print(x_train.info())
x_train, x_val , y_train, y_val = train_test_split(x_train,y_train['target'],test_size=0.2,stratify=y_train['target'])

model = RandomForestClassifier()
model.fit(x_train,y_train)

y_pred = model.predict(x_val)

print(y_pred)

f1 = f1_score(y_val,y_pred,average='macro')
print('f1 score 값 : ',f1 )

     pclass     sex   age  sibsp  parch   fare embarked    who  alone
3         1  female  35.0      1      0  53.10        S  woman  False
517       3    male   NaN      0      0  24.15        Q    man   True
861       2    male  21.0      1      0  11.50        S    man  False
487       1    male  58.0      0      0  29.70        C    man   True
58        2  female   5.0      1      2  27.75        S  child  False
<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 3 to 608
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   pclass      712 non-null    int64  
 1   age         712 non-null    float64
 2   sibsp       712 non-null    int64  
 3   parch       712 non-null    int64  
 4   fare        712 non-null    float64
 5   alone       712 non-null    bool   
 6   sex_female  712 non-null    uint8  
 7   sex_male    712 non-null    uint8  
 8   embarked_C  712 non-null    uint8  
 9   embarked_Q  712 non

[0 1 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 0 0
 0 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0
 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 1 0 0 1 1 1 1 0 1 0 1 1 0 0 0
 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 1 0 1 1 0 1 1 0 0]
f1 score 값 :  0.7931818181818182


In [1357]:
# 결과 제출 
y_result = model.predict(x_test)
result = pd.DataFrame({'cust_id':cust_id,'result':y_result})
# result.to_csv('생년월일_시험번호.csv',index=False)
result

,cust_id,result
0,800,0
1,341,1
2,413,0
3,575,0
4,202,0
...,...,...
174,402,0
175,70,0
176,53,1
177,743,0


In [1358]:
# 제3유형  단일표본 t 검정 
# mpg 데이터 평균이 20과 같은지 검정 
# 귀무가설 : 평균 =20 
# 대립가설 : 평균 != 20
# 정규성 검사 shapiro - 만족 : ttest_1samp
#                    - 만족 x : wilcoxon

s, p = shapiro(mtcars['mpg'])
print(p)
if p>0.05 : 
    print('정규성 만족')
else : 
    print('정규성 만족x')

0.12288373708724976
정규성 만족


In [1359]:
s,p = ttest_1samp(mtcars['mpg'],popmean=20)
print(p)
# 평균이 20과 같다
print(mtcars['mpg'].mean())

0.9327606409093872
20.090624999999996


In [1360]:
# 정규성 만족 x 
s,p = wilcoxon(mtcars['mpg']-20)
print(p)

0.7790732036774985


In [1361]:
# 문제 mpg 열 평균이 17보다 큰지 
# 귀무가설: 평균 <=17
# 대립가설: 평균 >17

s,p = ttest_1samp(mtcars['mpg'],popmean=17,alternative='greater')
print(p)
if p >0.05 :
    print('평균이 17보다 작다')
else :
    print('평균이 17보다 크다')

0.003394155007833913
평균이 17보다 크다


In [1362]:
# 문제 mpg 열 평균이 17보다 작은지 
# 귀무가설: 평균 >=17
# 대립가설: 평균 <17
s,p = ttest_1samp(mtcars['mpg'],popmean=17,alternative='less')
print(p)
if p >0.05 :
    print('평균이 17보다 크다')
else :
    print('평균이 17보다 작다')

0.9966058449921661
평균이 17보다 크다


In [1363]:
# 쌍체 t 검정 혈압약 먹기전후 같은지 
df = pd.DataFrame( {
    'before': [120, 135, 122, 124, 135, 122, 145, 160, 155, 142, 144, 135, 167],
    'after' : [110, 132, 123, 119, 123, 115, 140, 162, 142, 138, 135, 142, 160] })

df.head()

,before,after
0,120,110
1,135,132
2,122,123
3,124,119
4,135,123


In [1364]:
# 귀무가설 : 혈압약 먹기 전 후 같다
# 대립가설 : 혈압약 먹지 전 후 다르다
# 정규성 검정  shapiro 만족 : ttest_rel
#                     만족x : wilcoxon 

diff = df['after']-df['before']
s,p = shapiro(diff)
print(p)
if p >0.05:
    print('만족')
else:
    print('x')


0.7362736463546753
만족


In [1365]:
s, p = ttest_rel(df['after'],df['before'])
print(p)
if p >0.05 : 
    print('전 후 같음')
else: 
    print('전 후 다름')

0.008560014166295203
전 후 다름


In [1366]:
# 정규성 x 라면
s, p = wilcoxon(diff)
print(p)

0.013427734375


In [1367]:
# 독립표본 t tets
# A, B 그룹 혈압 평균 다른지 
df = pd.DataFrame({
  'A' : [120, 135, 122, 124, 135, 122, 145, 160, 155, 142, 144, 135, 167],
  'B' : [110, 132, 123, 119, 123, 115, 140, 162, 142, 138, 135, 142, 160]
})

print(df.head())

     A    B
0  120  110
1  135  132
2  122  123
3  124  119
4  135  123


In [1368]:
a_s, a_p = shapiro(df['A'])

b_s, b_p = shapiro(df['B'])

if (a_p >0.05)&(b_p>0.05):
    print('정규성 만족')
else : 
    print('정규성 만족x')


정규성 만족


In [1369]:
# 등분산성 확인 
eq_s, eq_p = bartlett(df['A'],df['B'])
print(eq_p)

0.867271716287991


In [1370]:
# a,b 그룹 둘다 정규성 만족시 -> ttest_ind 
#                     만족x  -> ranksums

s,p = ttest_ind(df['A'],df['B'])
print(p)

0.4207486213941666


In [1371]:
# 정규성 만족 안할때
r_s, r_p = ranksums(df['A'],df['B'])
print(r_p)

0.39746692542259277


In [1372]:
# a 그룹 혈압 평균이 b 그룹보다 큰지
# 귀무가설 : a 평균 <= b 평균
# 대립가설 : a 평균 > b 평균

t_s, t_p = ttest_ind(df['A'],df['B'],alternative='greater',equal_var=True)
print(t_p)

0.2103743106970833


In [1373]:
# anova 분석 
# a,b,c 세그룹 평균 같은지 검정
# 귀무가설 : a=b=c
# 대립가설 : 다르다 

# 정규성 검정 각각 그룹 모두 shapiro 검정 만족  -> f_oneway 
#                                       만족x -> kruskal

df = pd.DataFrame( {
    'A': [120, 135, 122, 124, 135, 122, 145, 160, 155, 142, 144, 135, 167],
    'B' : [110, 132, 123, 119, 123, 115, 140, 162, 142, 138, 135, 142, 160],
    'C' : [130, 120, 115, 122, 133, 144, 122, 120, 110, 134, 125, 122, 122]})

df.head()

,A,B,C
0,120,110,130
1,135,132,120
2,122,123,115
3,124,119,122
4,135,123,133


In [1374]:
eq_s, eq_p = bartlett(df['A'],df['B'],df['C'])
print(eq_p)

0.12110174433684852


In [1375]:
a_s, a_p = shapiro(df['A'])
b_s, b_p = shapiro(df['B'])
c_s, c_p = shapiro(df['C'])

print(a_p , b_p, c_p)

if (a_p >0.05)&(b_p>0.05)&(c_p>0.05):
    print('정규성 O')
else :
    print('정규성 x')

0.35584837198257446 0.5955665707588196 0.452654629945755
정규성 O


In [1376]:
s, p = f_oneway(df['A'],df['B'],df['C'])
print(p)
if p >0.05 : 
    print('세그룹 평균 동일')
else: 
    print('세그룹 평균 동일 x')

0.0346491232068625
세그룹 평균 동일 x


In [1377]:
# 정규성 x 라면
s,p = kruskal(df['A'],df['B'],df['C'])
print(p)
if p>0.05 : 
    print('세그룹 평균 동일')
else:
    print('세그룹 평균 동일 x')

0.03179333314266727
세그룹 평균 동일 x


In [1378]:
# chi  제곱 
# 랜덤박스 상품 a,b,c,d 있다. 랜덤 박스에서 100번 상품 꺼냈을때의 상품데이터라 할때 상품이 동일한 비율로 들어있는지 검정 
# 귀무가설: 동일한 비율
# 대립가설 : 동일한 비율 x 
row1 = [30, 20, 15, 35]

df = pd.DataFrame([row1], columns=['A','B','C','D'])

df

,A,B,C,D
0,30,20,15,35


In [1379]:
f_obs = [30,20,15,35]
f_exp = [100*0.25,100*0.25,100*0.25,100*0.25]
s,p =chisquare(f_obs,f_exp)
print(p)

if p>0.05:
    print('동일한비율')
else:
    print('동일한 비율 x')

0.01856613546304325
동일한 비율 x


In [1380]:
# 랜덤박스 상품 a,b,c 150번 상품 꺼냈을때 상품 데이터
# 상품별로 a 30%, b 15%, c 55% 인지 검증 
# 귀무가설 : 30%, 15%, 55% 비율로 들어있다
# 대립가설 : 아니다  
row1 = [50,25,75]

df= pd.DataFrame([row1],columns=['A','B','C'])


In [1381]:
f_obs = [50,25,75]
f_exp = [150*0.3, 150*0.15, 150*0.55]
s,p = chisquare(f_obs,f_exp)
if p >0.05 : 
    print('30%,15%,55% 비율')
else: 
    print('아니다')

30%,15%,55% 비율


In [1382]:
# 연령대에 따라 먹는 아이스크림 차이 있는지 독립성 검정 실시
# 귀무가설: 연령대와 먹는 아이스크림 종류 서로 관련 x
# 대립가설: 연령대와 먹는 아이스크림 종류 서로 관련 ㅇ
row1, row2 = [200, 190, 250], [220, 250, 300]

df = pd.DataFrame([row1, row2], columns=['딸기','초코','바닐라'], index=['10대','20대'])

df

,딸기,초코,바닐라
10대,200,190,250
20대,220,250,300


In [1383]:
s,p,d,e = chi2_contingency(df)
if p>0.05 : 
    print('관련 x')
else: 
    print('관련 o')

관련 x


In [1384]:
import seaborn as sns
df = sns.load_dataset('titanic')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [1385]:
# 셩별이랑 생존여부 간 독립성 검정 실시

df_1 = pd.crosstab(index=df['sex'],columns=df['survived'])
s,p,d,e = chi2_contingency(df_1)
if p>0.05 : 
    print('성별, 생존여부 관련 x')
else : 
    print('성별 생존여부 관련 o')

성별 생존여부 관련 o


In [1386]:
# 당뇨병 환자의 질병 진행정도  회귀식 구하기 
from sklearn.datasets import load_diabetes
diabets = load_diabetes()

x = pd.DataFrame(diabets.data, columns=diabets.feature_names)
y = pd.DataFrame(diabets.target)

y.columns=['target']

In [1387]:
x = x[['age','sex','bmi']]

In [1388]:
model = LinearRegression()
model.fit(x,y)

LinearRegression()

In [1389]:
# 결정계수 출력하기 r2

In [1390]:
# 회귀계수 출력
print(model.coef_)

print(model.coef_[0,0])
print(model.coef_[0,1])
print(model.coef_[0,2])

print(model.intercept_)



[[138.9039107  -36.13526678 926.91201212]]
138.9039106967436
-36.13526678248461
926.9120121197968
[152.13348416]


In [1391]:
import statsmodels.api as sm 
x = sm.add_constant(x)
model = sm.OLS(y,x).fit() 
summary = model.summary()
print(summary)

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.351
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     78.94
Date:                Sat, 30 Nov 2024   Prob (F-statistic):           7.77e-41
Time:                        01:05:42   Log-Likelihood:                -2451.6
No. Observations:                 442   AIC:                             4911.
Df Residuals:                     438   BIC:                             4928.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        152.1335      2.964     51.321      0.0

In [1392]:
diabetes = load_diabetes()

x = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = pd.DataFrame(diabetes.target)
y.columns = ['target']

print(x.head())
print(y.head())

        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005671 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6  
0 -0.002592  0.019908 -0.017646  
1 -0.039493 -0.068330 -0.092204  
2 -0.002592  0.002864 -0.025930  
3  0.034309  0.022692 -0.009362  
4 -0.002592 -0.031991 -0.046641  
   target
0   151.0
1    75.0
2   141.0
3   206.0
4   135.0


In [1393]:
# 상관분석 진행할 독립 변수, 종속변수 선정 
x = x['bmi']
y = y['target']
# 귀무가설: 두 변수간 선형관계 존재x
# 대립가설: 두 변수간 선형관계 존재 
r,p = pearsonr(x,y)
print(p)
print(r)
if p>0.05 : 
    print('두변수간 상관관계 존재 하지 않음')
else :
    print('상관관계 존재 한다')

3.4660064451655095e-42
0.5864501344746886
상관관계 존재 한다


In [1394]:
# 검정 통계량 
n = len(x)
r2 = r**2 
r*(n-2)**0.5/(1-r2)**0.5

15.187289570365305

In [1395]:
# 로지스틱 회귀 분석
import seaborn as sns
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [1396]:
# 분석 데이터 결정 
# 독립변수 : sex, sibsp, fare
# 종속변수 : survivied 
x = df[['sex','sibsp','fare']]
y= df[['survived']]

In [1397]:
# 변수가 문자형인 경우 변환 필요
# LabelEncoder은 사전에 정의된 순서대로 0,1,2 변환
# 따라서 female : 0 ,male :1 
# le = LabelEncoder()
# x['sex'] = le.fit_transform(x['sex'])
# x.head()

# 만약 female :1 male :0 하고 싶다면
x['sex'] = x['sex'].map({'female':1,'male':0})
x.head()

C:\Users\chohyeri\AppData\Local\Temp\ipykernel_8648\2501839624.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['sex'] = x['sex'].map({'female':1,'male':0})


,sex,sibsp,fare
0,0,1,7.2500
1,1,1,71.2833
2,1,0,7.9250
3,1,1,53.1000
4,0,0,8.0500


In [1398]:
model = LogisticRegression(penalty='none')
model.fit(x,y)
print(model.coef_)

[[ 2.56677766 -0.40170461  0.01375665]]


c:\Users\chohyeri\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [1399]:
model.intercept_

array([-1.69644546])

In [1400]:
# 로지스틱 회귀 모형에서 sibsp 변수 한단위 증가 할때 생존할 오즈 몇배?
odds_ratio = round(np.exp(model.coef_[0,1]),3)
print(odds_ratio)

0.669


In [1401]:
odds_ratio = round(np.exp(model.coef_[0,0]),3)
odds_ratio

13.024

In [1402]:
import statsmodels.api as sm
x= sm.add_constant(x)

model = sm.Logit(y,x).fit()
summary = model.summary()
print(summary)

Optimization terminated successfully.
         Current function value: 0.483846
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      887
Method:                           MLE   Df Model:                            3
Date:                Sat, 30 Nov 2024   Pseudo R-squ.:                  0.2734
Time:                        01:05:43   Log-Likelihood:                -431.11
converged:                       True   LL-Null:                       -593.33
Covariance Type:            nonrobust   LLR p-value:                 5.094e-70
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.6964      0.129    -13.134      0.000      -1.950      -1.443
sex            2.5668      0.